<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/Spediteur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# pip als Paketmanager
! pip install -q pyscipopt

In [24]:
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Produktionsplanung

In [25]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmengen

$i \in \mathcal{I}$

In [26]:
I = ['A1','A2','A3','A4','A5','A6'] # Menge der Aufträge

## Parameter (Daten)

$db_i$, $v_i$, $lv$

In [27]:
db={} # Deckungsbeiträge
db['A1']=10
db['A2']=15
db['A3']=25
db['A4']=5
db['A5']=11
db['A6']=8

In [28]:
# Auftragsvolumen (m^3)
v={}
v['A1']=3
v['A2']=4.5
v['A3']=9.25
v['A4']=1.2
v['A5']=4
v['A6']=2.7

In [29]:
lv=10 # Ladevolumen LKW

## Entscheidungsvariablen

$X_i \in \{0,1\}$

In [30]:
# Definition der Entscheidungsvariablen
x={}
for i in I:
    x[i] = scip.addVar(vtype='B', name=f"X_{i}")
print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [X_A1, X_A2, X_A3, X_A4, X_A5, X_A6]


## Zielfunktion

$\text{max} DB = \sum_i db_i \cdot X_i$

In [31]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(db[i]*x[i] for i in I), sense="maximize")

## Nebenbedingungen/ Restriktionen

$\sum_i v_i \cdot X_i \leq lv$

In [32]:
scip.addCons(quicksum(v[i]*x[i] for i in I)<=lv, name="Laderaumvolumen")

Laderaumvolumen

In [33]:
print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Laderaumvolumen]


## Berechnung der Lösung

In [34]:
scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', round(scip.getObjVal()))
    volumen_used=0
    for i in I:
        print(f'{i} =', round(scip.getVal(x[i])))
        volumen_used += v[i]*round(scip.getVal(x[i]))
    print(f"Ladevolumen: {volumen_used} m^3")
else:
    print('Problem hat keine Lösung')

Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 31
A1 = 0
A2 = 1
A3 = 0
A4 = 1
A5 = 1
A6 = 0
Ladevolumen: 9.7 m^3
